In [2]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt

In [3]:
IMAGE_SIZE = 256
BATCH_SIZE = 64
CHANNELS = 3
EPOCHS = 35
n_classes = 10

tf.config.get_visible_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "Tomatoes",
    shuffle=True,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
)

Found 16011 files belonging to 10 classes.


In [5]:
class_names = dataset.class_names
print(class_names)

['Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_Leaf_Mold', 'Tomato_Septoria_leaf_spot', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato__Target_Spot', 'Tomato__Tomato_YellowLeaf__Curl_Virus', 'Tomato__Tomato_mosaic_virus', 'Tomato_healthy']


In [6]:
def get_dataset_partitions_tf(
    ds, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000
):
    ds_size = len(ds)
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)

    if shuffle:
        ds.shuffle(shuffle_size, seed=22)

    train_ds = ds.take(train_size)
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size + val_size)

    return train_ds, val_ds, test_ds

In [7]:
train_ds, val_ds, test_ds = get_dataset_partitions_tf(dataset)
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
# caches images for next epoch, prefetch fetches next batch before next iteration

In [8]:
resize_and_rescale = tf.keras.Sequential(
    [
        layers.experimental.preprocessing.Resizing(IMAGE_SIZE, IMAGE_SIZE),
        layers.experimental.preprocessing.Rescaling(1.0 / 255),
    ]
)

In [9]:
data_augmentation = tf.keras.Sequential(
        [
            layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
            layers.experimental.preprocessing.RandomRotation(0.2),
        ]
    )

In [10]:
input_shape = (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, CHANNELS)

In [11]:
model = models.Sequential(
    [
        resize_and_rescale,
        data_augmentation,
        layers.Conv2D(BATCH_SIZE, (3, 3), activation="relu", input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.BatchNormalization(),
        layers.Conv2D(
            64,
            (3, 3),
            activation="relu",
            padding="same",
        ),
        layers.MaxPooling2D((2, 2)),
        layers.BatchNormalization(),
        layers.Conv2D(
            64,
            (3, 3),
            activation="relu",
            padding="same",
        ),
        layers.MaxPooling2D((2, 2)),
        layers.BatchNormalization(),
        layers.Conv2D(
            64,
            (3, 3),
            activation="relu",
            padding="same",
        ),
        layers.MaxPooling2D((2, 2)),
        layers.BatchNormalization(),
        layers.Conv2D(
            64,
            (3, 3),
            activation="relu",
            padding="same",
        ),
        layers.MaxPooling2D((2, 2)),
        layers.BatchNormalization(),
        layers.Conv2D(
            64,
            (3, 3),
            activation="relu",
            padding="same",
        ),
        layers.MaxPooling2D((2, 2)),
        layers.BatchNormalization(),
        layers.Conv2D(
            64,
            (3, 3),
            activation="relu",
            padding="same",
        ),
        layers.MaxPooling2D((2, 2)),
        layers.BatchNormalization(),
        layers.Flatten(),
        layers.Dense(64, activation="relu"),
        layers.Dense(n_classes, activation='softmax'),
    ]
)

In [12]:
model.build(input_shape=input_shape)

In [13]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=10000,
    decay_rate=0.1,
   )

model.compile(
    optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=lr_schedule),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=["accuracy"],
)

In [14]:
history = model.fit(
    train_ds,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose=1,
    validation_data=val_ds,
)

Epoch 1/35
200/200 [==============================] - 109s 513ms/step - loss: 0.8933 - accuracy: 0.7007 - val_loss: 3.9463 - val_accuracy: 0.1663
Epoch 2/35
200/200 [==============================] - 106s 531ms/step - loss: 0.4774 - accuracy: 0.8356 - val_loss: 3.7777 - val_accuracy: 0.2463
Epoch 3/35
200/200 [==============================] - 107s 535ms/step - loss: 0.3775 - accuracy: 0.8715 - val_loss: 1.6151 - val_accuracy: 0.5975
Epoch 4/35
200/200 [==============================] - 102s 512ms/step - loss: 0.2970 - accuracy: 0.9016 - val_loss: 1.2420 - val_accuracy: 0.7056
Epoch 5/35
200/200 [==============================] - 110s 550ms/step - loss: 0.2482 - accuracy: 0.9147 - val_loss: 2.5694 - val_accuracy: 0.4881
Epoch 6/35
200/200 [==============================] - 104s 520ms/step - loss: 0.2309 - accuracy: 0.9206 - val_loss: 2.5861 - val_accuracy: 0.5494
Epoch 7/35
200/200 [==============================] - 107s 537ms/step - loss: 0.2012 - accuracy: 0.9309 - val_loss: 0.5611 -

In [15]:
scores = model.evaluate(test_ds)

26/26 [==============================] - 4s 68ms/step - loss: 0.2158 - accuracy: 0.9392


In [16]:
model.save("../model/4")

INFO:tensorflow:Assets written to: ../model/4/assets


INFO:tensorflow:Assets written to: ../model/4/assets
